# Ontology ancestors mapping
This notebook can be used to compute mappings between nodes in an ontology files and their ancestors.
It will:
1. Load the required ontology files (human, mouse, UBERON)
1. For `human` and `mouse`, start from each node available in the ontology
1. for `UBERON`, only start from a defined, hardcoded subset of leaf nodes which is relevant for our datasets
1. Recursively compute ancestors using the `is_a` relationship and `part_of` restriction (defined as `BFO_0000050`)
1. Create mappings from each node to its ancestors
1. Write a json file with these mappings (a dictionary of lists)

In [ ]:
from owlready2 import *
import json

In [ ]:
def get_ancestors(class_name):

    z = onto.search_one(iri = f'http://purl.obolibrary.org/obo/{class_name}')

    def recurse(x):
        for e in x.is_a:
            if hasattr(e, "value"):
                prop = e.property.name
                if prop != "BFO_0000050": continue
                val = e.value.name.replace("obo.", "")
                z = onto.search_one(iri = f'http://purl.obolibrary.org/obo/{val}')
                yield (z, z.name, z.label, z.IAO_0000115)
                yield from recurse(z)


    yield (z, z.name, z.label, z.IAO_0000115)
    yield from recurse(z)

In [ ]:
def create_mapping(classes, prefix = None):
    x = dict()
    for cls in classes:
        for a in get_ancestors(cls.name):
            if prefix and not a[1].startswith(prefix):
                continue
            if cls.name in x:
                x[cls.name].append(a[1])
            else:
                x[cls.name] = [a[1]]    
    return x

In [ ]:
human_ontology = "http://purl.obolibrary.org/obo/hsapdv.owl"

onto = get_ontology(human_ontology)
onto.load()

m_human = create_mapping(onto.classes(), "HsapDv")

In [ ]:
mouse_ontology = "http://purl.obolibrary.org/obo/mmusdv.owl"

onto = get_ontology(mouse_ontology)
onto.load()

m_mouse = create_mapping(onto.classes(), "MmusDv")

In [ ]:
uberon_ontology = "http://purl.obolibrary.org/obo/uberon.owl"

onto = get_ontology(uberon_ontology)
onto.load()

In [ ]:
uberon_classes = [
    "UBERON_0007236",
    "UBERON_0000106",
    "UBERON_0014859",
    "UBERON_0008264",
    "UBERON_0007233",
    "UBERON_0000112",
    "UBERON_8000003",
    "UBERON_0014857",
    "UBERON_0009849",
    "UBERON_0034920",
    "UBERON_0000069",
    "UBERON_0000109",
    "UBERON_8000001",
    "UBERON_0000068",
    "UBERON_0018685",
    "UBERON_0000107",
    "UBERON_0007222",
    "UBERON_0000092",
    "UBERON_0018378",
    "UBERON_0014864",
    "UBERON_0004730",
    "UBERON_0000111",
    "UBERON_0007220",
    "UBERON_0014405",
    "UBERON_0014862",
    "UBERON_8000000",
    "UBERON_0000071",
    "UBERON_0014860",
    "UBERON_0012101",
    "UBERON_0000113",
    "UBERON_0014858",
    "UBERON_0007232",
    "UBERON_0000070",
    "UBERON_0000110",
    "UBERON_8000002",
    "UBERON_0014856",
    "UBERON_0004728",
    "UBERON_0034919",
    "UBERON_0000108",
    "UBERON_0000066",
    "UBERON_0004707",
    "UBERON_0000105",
    "UBERON_0018241",
    "UBERON_0007221",
    "UBERON_0014406",
    "UBERON_0014863",
    "UBERON_0004729",
    "UBERON_0014861",
]

In [ ]:
uc = [onto.search_one(iri = f'http://purl.obolibrary.org/obo/{c}') for c in uberon_classes]

m_uberon = create_mapping(uc, "UBERON")

In [ ]:
with open("../backend/corpora/common/utils/ontology_mappings/fixtures/development_stage_ontology_mapping.json", "w") as f:
    d = {}
    d.update(m_human)
    d.update(m_mouse)
    d.update(m_uberon)
    json.dump(d, f)